In [ ]:
import numpy as np
from keras.models import Model, load_model
from keras import backend as K

import gc

In [2]:
baseModelPath  = './baseModel/modelBasev3.004-0.73.hdf5'
baseModel = load_model(baseModelPath, compile=False)

modelPath = './models/m2p01_block2_conv1/modelTuned.block2_conv1.001-022.hdf5'
model = load_model(modelPath, compile=False)

In [60]:
# Note that xTrain and xVal are normalized by /=255
#xTrain = np.load('./data/xTrain.npy', mmap_mode='r')
#yTrain = np.load('./data/yTrain.npy', mmap_mode='r')
xVal   = np.load('./data/xVal.npy', mmap_mode='r')
yVal   = np.load('./data/yVal.npy', mmap_mode='r')

import pickle
with open("./data/xValFileNames.txt", "rb") as fp:
  xValFileNames = pickle.load(fp)
xValFileNames = np.array(xValFileNames)
categories = ['cat','dog','horse','person']
datasets = ['train','val','test']

In [3]:
predBase = np.load('./baseModel/modelBasev3.004-0.73_pred.npy')
pred = np.load('./models/m2p01_block2_conv1/modelTuned.block2_conv1.001-022_pred.npy')

In [64]:
# Get indices of top N imgs that changed prediction for each class
# L1 DISTANCE
N=3

topNIdx = np.zeros((0,4), dtype=np.int)
predChange = np.abs(pred - predBase)
pred_tmp = predChange

for n in range(N):
    idx = np.argmax(pred_tmp,axis=0)
    topNIdx = np.append(arr=topNIdx,axis=0,values=idx.reshape((1,4)))
    pred_tmp = np.delete(pred_tmp, idx,axis=0)

In [131]:
topN = xValFileNames[topNIdx]

# GradCAM the top changed imgs

In [93]:
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from keras import backend as K
from keras.preprocessing import image

import tensorflow as tf
from tensorflow.python.framework import ops

from imageio import imread, mimsave

In [94]:
imgSize=128
H = imgSize
W = imgSize

N_CLASS = 4

### Utility functions

In [95]:
def build_model(modelPath):
  """Function returning keras model instance.
  """
  model = load_model(modelPath, compile=False)
  return model

In [96]:
def load_image(path, preprocess=True):
  """Load and preprocess image."""
  if preprocess:
    x = np.array(imread(path),dtype=np.float32)
    x = np.expand_dims(x, axis=0)
    #print(x)
    x/=255. # This was the only preprocessing I did... rather than keras.applications.preprocess_input(x)
  else:
    x = image.load_img(path, target_size=(H, W))
  return x

def deprocess_image(x):
  """Same normalization as in:
  https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
  """
  x = x.copy()
  if np.ndim(x) > 3:
    x = np.squeeze(x)
  # normalize tensor: center on 0., ensure std is 0.1
  x -= x.mean()
  x /= (x.std() + 1e-5)
  x *= 0.1

  # clip to [0, 1]
  x += 0.5
  x = np.clip(x, 0, 1)

  # convert to RGB array
  x *= 255
  if K.image_dim_ordering() == 'th':
    x = x.transpose((1, 2, 0))
  x = np.clip(x, 0, 255).astype('uint8')
  return x


def normalize(x):
  """Utility function to normalize a tensor by its L2 norm"""
  return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)

In [97]:
import json
from keras.utils.data_utils import get_file
from keras import backend as K

CLASS_INDEX = json.load(open('./myModel_class_index.json'))
    
def decode_predictions(preds, top=4):
    """Decodes the prediction of ourModel.
    # Arguments
        preds: Numpy tensor encoding a batch of predictions.
        top: Integer, how many top-guesses to return.
    # Returns
        A list of lists of top class prediction tuples
        `(class_name, class_description, score)`.
        One list of tuples per sample in batch input.
    # Raises
        ValueError: In case of invalid shape of the `pred` array
            (must be 2D).
    # Note: this is a modification of keras.applications.imagenet_utils.preprocess_input
    """
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 4:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 4)). '
                         'Found array with shape: ' + str(preds.shape))
    
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [98]:
def build_guided_model(modelPath):
    """Function returning modified model.
    
    Changes gradient function for all ReLu activations
    according to Guided Backpropagation.
    """
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                   tf.cast(op.inputs[0] > 0., dtype)

    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': 'GuidedBackProp'}):
        new_model = build_model(modelPath)
    return new_model


def guided_backprop(input_model, images, layer_name):
    """Guided Backpropagation method for visualizing input saliency."""
    input_imgs = input_model.input
    layer_output = input_model.get_layer(layer_name).output
    grads = K.gradients(layer_output, input_imgs)[0]
    backprop_fn = K.function([input_imgs, K.learning_phase()], [grads])
    grads_val = backprop_fn([images, 0])[0]
    return grads_val

In [99]:
def grad_cam(input_model, image, target_cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, target_cls]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize if necessary
    # grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (H, W), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

## Step2 of GradCAM the top changed imgs in [val set]

In [112]:
# Make directories if not already exist
categories = ['cat','dog','horse','person']
imgLists = [img_cat, img_dog, img_horse, img_person]

srcDir = './data/val/'
targetDir = './CAMs/predchgs/'

for f1 in ['GradCAM', 'G_GradCAM', 'G_BackProp']:
  p1 = '{}{}/'.format(targetDir,f1)
  if not os.path.exists(p1):
    #print('{} path does not exist'.format(p1))
    os.makedirs(p1)
  for categ in categories:
    p2='{}{}'.format(p1,categ)
    if not os.path.exists(p2):
      #print('{} path does not exist'.format(p2))
      os.makedirs(p2)

In [145]:
print(topN)

[['2008_004538.jpg' '2008_001531.jpg' '2008_001359.jpg' '2008_004538.jpg']
 ['2008_003374.jpg' '2008_004584.jpg' '2008_003975.jpg' '2008_001454.jpg']
 ['2008_002234.jpg' '2008_001789.jpg' '2008_005313.jpg' '2008_001205.jpg']]


In [143]:
topNIdx

array([[ 10, 633, 582,  10],
       [179, 310, 194, 610],
       [ 21, 721, 317, 533]], dtype=int64)

In [133]:
for imgList in topN:
    for img in imgList:
        for categ in categories:
            p1 = '{}{}/{}'.format(srcDir,categ,img)
            if not os.path.exists(p1):
                continue
            print('{} exist'.format(p1))

./data/val/cat/2008_004538.jpg exist
./data/val/person/2008_001531.jpg exist
./data/val/person/2008_001359.jpg exist
./data/val/cat/2008_004538.jpg exist
./data/val/dog/2008_003374.jpg exist
./data/val/horse/2008_004584.jpg exist
./data/val/dog/2008_003975.jpg exist
./data/val/person/2008_001454.jpg exist
./data/val/cat/2008_002234.jpg exist
./data/val/person/2008_001789.jpg exist
./data/val/horse/2008_005313.jpg exist
./data/val/person/2008_001205.jpg exist


In [ ]:
# GENERATE CAMs FOR BASELINE MODEL
layerName = 'block5_conv3'
idx = 0
chPruned = 0
prunedLayer='base'
prunedIdx='000'

# NOTE: Project 'm2p01' is the right project for channel pruning
baseModelPath  = './baseModel/modelBasev3.004-0.73.hdf5'
baseModel = build_model(baseModelPath)
guided_baseModel = build_guided_model(baseModelPath)

#######################################################################
for categ in categories:
  for srcImgs in topN:
    for srcImg in srcImgs:
    
      # Load npy img
      imgPath = '{}{}/{}'.format(srcDir,categ,srcImg)
      if not os.path.exists(imgPath):
          continue
#######################################################################
      
      img =load_image(imgPath)
      pred_base = np.round(baseModel.predict(img)[0], decimals=2)
      pred_base = np.array2string(pred_base,separator=',', formatter={'float_kind':lambda x: '{:.2f}'.format(x)})
      pred_base = pred_base.replace('[','(').replace(']',')')
  
      # Perform {GradCAM, etc.} to explain all target_cls and save images
      for target_cls in [0,1,2,3]:
        p_or_t = 'b'
        imgName = '{}_{}_{:03d}{}_{}_{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,prunedLayer, prunedIdx, chPruned, pred_base)
        #imgName = '{}_{}_{:03d}{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,chPruned,pred_base)
  
        targetPath_gc  = '{}GradCAM/{}/{}'.format(targetDir,categ, imgName)
        targetPath_ggc = '{}G_GradCAM/{}/{}'.format(targetDir,categ, imgName)
        #targetPath_gb  = '{}G_BackProp/{}/{}'.format(targetDir,categ, imgName)
  
        gradcam = grad_cam(baseModel, img, target_cls, layerName)
        gb = guided_backprop(guided_baseModel, img, layerName)
        guided_gradcam = gb * gradcam[..., np.newaxis]
  
        jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
        jetcam = (np.float32(jetcam) + load_image(imgPath, preprocess=False)) / 2
  
        cv2.imwrite(targetPath_gc, np.uint8(jetcam))
        #cv2.imwrite(targetPath_gb, deprocess_image(gb[0]))
        cv2.imwrite(targetPath_ggc, deprocess_image(guided_gradcam[0]))   

## [val set] largest pred change: pruned-tuned

In [141]:
import pickle
with open("./data/cmds.txt", "rb") as fp:
  cmds = pickle.load(fp)

del pickle

In [ ]:
#################################
contFromCmd = 0
idx = contFromCmd*2+1
#################################

for i,cmd in enumerate(cmds[contFromCmd:], contFromCmd):
  # Load prunedModel, tunedModel, guided_prunedModel, guided_tunedModel
  # AND load chPruned, prunedLayer, prunedIdx
  print('Loading models [cmd {}]'.format(i))
  K.clear_session()
  exec(cmd)
  print('===')
  
  #######################################################################
  for categ in categories:
    for srcImgs in topN:
      for srcImg in srcImgs:
      
      
        # Load npy img
        imgPath = '{}{}/{}'.format(srcDir,categ,srcImg)
        if not os.path.exists(imgPath):
            continue
  #######################################################################
      
        # Load npy img
        #imgPath = '{}{}/{}.jpg'.format(srcDir,categ,srcImg)
        img =load_image(imgPath)
        pred_pruned = np.round(prunedModel.predict(img)[0], decimals=2)
        pred_pruned = np.array2string(pred_pruned,separator=',', formatter={'float_kind':lambda x: '{:.2f}'.format(x)})
        pred_pruned = pred_pruned.replace('[','(').replace(']',')')
        
        pred_tuned  = np.round(tunedModel.predict(img)[0], decimals=2)
        pred_tuned  = np.array2string(pred_tuned,separator=',', formatter={'float_kind':lambda x: '{:.2f}'.format(x)})
        pred_tuned  = pred_tuned.replace('[','(').replace(']',')')
        
        # Perform GradCAM to explain all target_cls and save images
        for target_cls in [0,1,2,3]:
          p_or_t = 'p'
          imgName = '{}_{}_{:03d}{}_{}_{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,prunedLayer, prunedIdx, chPruned,pred_pruned)
          
          targetPath_gc  = '{}GradCAM/{}/{}'.format(targetDir,categ, imgName)
          targetPath_ggc = '{}G_GradCAM/{}/{}'.format(targetDir,categ, imgName)
          #targetPath_gb  = '{}G_BackProp/{}/{}'.format(targetDir,categ, imgName)
          
          gradcam = grad_cam(prunedModel, img, target_cls, layerName)
          gb = guided_backprop(guided_prunedModel, img, layerName)
          guided_gradcam = gb * gradcam[..., np.newaxis]
  
          jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
          jetcam = (np.float32(jetcam) + load_image(imgPath, preprocess=False)) / 2
  
          cv2.imwrite(targetPath_gc, np.uint8(jetcam))
          #cv2.imwrite(targetPath_gb, deprocess_image(gb[0]))
          cv2.imwrite(targetPath_ggc, deprocess_image(guided_gradcam[0]))
        print('{}: {}'.format(idx,imgName))
        idx+=1
        
        for target_cls in [0,1,2,3]:
          p_or_t = 't'
          imgName = '{}_{}_{:03d}{}_{}_{}_{}_{}.jpg'.format(srcImg,target_cls,idx,p_or_t,prunedLayer, prunedIdx, chPruned,pred_tuned)
  
          targetPath_gc  = '{}GradCAM/{}/{}'.format(targetDir,categ, imgName)
          targetPath_ggc = '{}G_GradCAM/{}/{}'.format(targetDir,categ, imgName)
          #targetPath_gb  = '{}G_BackProp/{}/{}'.format(targetDir,categ, imgName)
  
          gradcam = grad_cam(tunedModel, img, target_cls, layerName)
          gb = guided_backprop(guided_tunedModel, img, layerName)
          guided_gradcam = gb * gradcam[..., np.newaxis]
  
          jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
          jetcam = (np.float32(jetcam) + load_image(imgPath, preprocess=False)) / 2
  
          cv2.imwrite(targetPath_gc, np.uint8(jetcam))
          #cv2.imwrite(targetPath_gb, deprocess_image(gb[0]))
          cv2.imwrite(targetPath_ggc, deprocess_image(guided_gradcam[0]))
        print('{}: {}'.format(idx,imgName))
        idx-=1
  idx+=2